# 07장 BJT 증폭기

* 이름: 홍길동
* 학번: 2024123456
* 조: 1

## 공통 코드

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import io
import os
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from scipy import interpolate
from numpy import nan, inf

from IPython.display import Image

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def tangent_line_end(f, x):
    h = 1e-4
    d = (f(x) - f(x-h)) / (h)
    return lambda t: d*t - d*x + f(x)

def tangent_line_and_gradient(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return (lambda t: d*t - d*x + f(x)), d

def tangent_line_and_gradient_end(f, x):
    h = 1e-4
    d = (f(x) - f(x-h)) / (h)
    return (lambda t: d*t - d*x + f(x)), d

def derivative(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return d

def get_simulation_result(file_name, start=0, end=-1):
    if not os.path.exists(file_name):
        print("{} doesn't exist.".format(file_name))
        return None

    if end == -1:
        end = None
    else:
        end += 1

    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []

            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]

            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    if not os.path.exists(file_name):
        print("{} doesn't exist.".format(file_name))
        return None

    if end == -1:
        end = None
    else:
        end += 1

    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def print_value_to_string(label, value):
    output = io.StringIO()
    print('%s = ' % label, end='', file=output)
    print('{:11.3f}'.format(value), file=output)
    captured = output.getvalue()
    return captured

def draw_plot(xs, ys, label, style_idx, color_idx=-1, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if color_idx < 0:
        color_idx = style_idx

    if isinstance(xs, list):
        xs = np.array(xs)
    if isinstance(ys, list):
        ys = np.array(ys)

    if (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    elif not isinstance(xs, np.ndarray):
        selected_markevery = None
        selected_marker = markers[style_idx]
    elif (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[color_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[color_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []


def calculate_square_wave_frequency(times, vins):
    # 전압 값의 중간값을 임계값으로 설정
    threshold = (np.max(vins) + np.min(vins)) / 2

    # 상승 에지 감지
    rising_edges = np.where((vins[:-1] < threshold) & (vins[1:] >= threshold))[0]

    if len(rising_edges) < 2:
        return None  # 주기를 계산할 수 없음

    # 주기 계산 (연속된 상승 에지 사이의 평균 시간)
    periods = times[rising_edges[1:]] - times[rising_edges[:-1]]
    average_period = np.mean(periods)

    # 주파수 계산 (주기의 역수)
    frequency = 1 / average_period

    return frequency

def display_image(file_path, width=None, height=None):
    if os.path.exists(file_path):
        display(Image(file_path, width=width, height=height))
    else:
        print("{} doesn't exist.".format(file_path))

def find_first_peak_index(xs, ys, margine=10):
    xs_len = len(xs)
    old_grad = None
    pi = -1
    m = margine
    for i in range(xs_len - 1):
        grad = (ys[i+1] - ys[i]) / (xs[i+1] - xs[i])
        if grad > 0:
            pi = i
            m -= 1
            if m == 0:
                old_grad = grad
                break
        else:
            m = margine
    if old_grad != None:
        m = margine
        for i in range(pi, xs_len - 1):
            grad = (ys[i+1] - ys[i]) / (xs[i+1] - xs[i])
            if grad < 0:
                pi = i
                m -= 1
                if m == 0:
                    pi -= margine
                    break
            else:
                m = margine
    return pi


*******************************************************************************
## 실험01 $\beta$값의 측정




### 실험01-01 $\beta$ 측정

#### 실험 방법

* 멀티미터 DM332의 $\beta$ (hFE) 측정기능을 사용해 NPN형 트랜지스터인 2N2222의 $\beta$ 값을 측정한다.

In [ ]:
display_image("./images/ch07_ep01_01_tr_01.png", width=100)
display_image("./images/ch07_ep01_01_dmm_01.png", width=400)

#### 실험 결과

$\beta$ 값: ?? 216

*******************************************************************************
## 실험02 비선형 전달특성을 가지는 증폭기

### 실험02-01 회로의 구성

#### 실험 방법

* 트랜지스터: 2N2222

In [ ]:
display_image("./images/ch07_ep01_02_circuit.png", width=300)

#### 실험 결과

##### 제작한 회로

In [ ]:
display_image("./data/ch07_ep01_02_result_circuit.jpg", width=1000)

### 실험02-02 파형발생기로 실험

#### 실험 방법

* Vi: 
    + 정현파, 주파수 1 kHz, 크기 2 Vpp
    + 정현파, 주파수 50 Hz, 크기 7 Vpp

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch07_ep02_02_result_1khz_2vpp.png", width=600)
display_image("./data/ch07_ep02_02_result_50hz_7vpp.png", width=600)
display_image("./data/ch07_ep02_02_result_50hz_7vpp_xy.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep02_02_result_1khz_2vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

vout_dc = np.mean(vouts)
vouts = vouts - vout_dc

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.ylim(-2, 2)
plt.show()


In [ ]:
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep02_02_result_50hz_7vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

vout_dc = np.mean(vouts)
vouts = vouts - vout_dc

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.ylim(-11, 11)
plt.show()

# 입력 전압을 x축, 출력 전압을 y축으로 하는 그래프를 그린다.
plt.xlabel('V_IN_V')
plt.ylabel('V_OUT_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vins[10:-10], vouts[10:-10], "XY", 0)
plt.show()


*******************************************************************************
## 실험03 CE증폭기회로1의 동작점

### 실험03-01 회로의 구성

#### 실험 방법

* 트랜지스터: 2N2222

In [ ]:
display_image("./images/ch07_ep03_01_circuit.png", width=300)

#### 실험 결과

##### 제작한 회로

In [ ]:
display_image("./data/ch07_ep03_01_result_circuit.jpg", width=1000)

### 실험03-02 동작점 측정

#### 실험 방법

* V_I, V_O 를 연결하지 않은 상태에서 V_C, V_E를 측정한 후 그 값으로 V_CE와 I_C를 계산한다.

#### 실험 결과

#### 결과 분석

In [ ]:
# R1 100 kOhm
# R2 22 kOhm

V_CC_V = 15.0
R_C_Ohm = 3000.0

V_C_V = ??
V_E_V = ??

V_CE_V = V_C_V - V_E_V
I_C_mA = (V_CC_V - V_C_V) / R_C_Ohm * 1000

print_value('I_C_mA', I_C_mA)
print_value('V_CE_V', V_CE_V)

*******************************************************************************
## 실험04 CE증폭기회로1의 동작점의 민감도 실험

### 실험04-01 동작점 이동 관측

#### 실험 방법

* R1에 100 kOhm 대신 68 kOhm을 연결한 다음 동작점 측정
* R2에 22 kOhm 대신 15 kOhm을 연결한 다음 동작점 측정

#### 실험 결과

#### 결과 분석

In [ ]:
# R1 68 kOhm
# R2 22 kOhm

V_CC_V = 15.0
R_C_Ohm = 3000.0

V_C_V = ??
V_E_V = ??

V_CE_V = V_C_V - V_E_V
I_C_mA = (V_CC_V - V_C_V) / R_C_Ohm * 1000

print_value('I_C_mA', I_C_mA)
print_value('V_CE_V', V_CE_V)

In [ ]:
# R1 100 kOhm
# R2 15 kOhm

V_CC_V = 15.0
R_C_Ohm = 3000.0

V_C_V = ??
V_E_V = ??

V_CE_V = V_C_V - V_E_V
I_C_mA = (V_CC_V - V_C_V) / R_C_Ohm * 1000

print_value('I_C_mA', I_C_mA)
print_value('V_CE_V', V_CE_V)

*******************************************************************************
## 실험05 CE증폭기회로1의 증폭실험

### 실험05-01 입출력 측정

#### 실험 방법

* Vi: 
    + 정현파, 주파수 1 kHz, 크기 2 Vpp
    + 출력을 측정하는 오실로스코프 CH2의 커플링을 AC모드로 변환

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch07_ep05_02_result_1khz_2vpp.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_1khz_2vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# vout과 동일한 주파수의 싸인파 곡선을 추가한다.
frequency = 1000
amplitude = (np.max(vouts) - np.min(vouts)) / 2
sine_wave = -np.sin(2 * np.pi * frequency * times) * amplitude

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, sine_wave, 'Sine Wave', 2)
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.show()

# Calculate the difference between the sine wave and vout
difference = np.abs(sine_wave - vouts)
# Calculate the mean difference
mean_difference_mV = np.mean(difference) * 1000
print_value('mean_difference_mV', mean_difference_mV)

print()
V_I_pp_V = vins.max() - vins.min()
V_O_pp_V = vouts.max() - vouts.min()
A_V_measured = - V_O_pp_V / V_I_pp_V
print_value("V_I_pp_V", V_I_pp_V)
print_value("V_O_pp_V", V_O_pp_V)
print_value("A_V_measured", A_V_measured)

print()
R_E_Ohm = 1000
R_C_Ohm = 3000
A_V_theoretical = - R_C_Ohm / R_E_Ohm
print_value("A_V_theoretical", A_V_theoretical)

### 실험04-02 왜곡 관측

#### 실험 방법

* Vi의 크기를 변경해 왜곡이 나타나지 않는 시작 전압을 관측한다.
    + 값은 소수점 한자리 단위로 변경

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch07_ep05_02_result_1khz_4vpp.png", width=600)
display_image("./data/ch07_ep05_02_result_1khz_3_9vpp.png", width=600)
display_image("./data/ch07_ep05_02_result_1khz_3_8vpp.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_1khz_4vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# vout과 동일한 주파수의 싸인파 곡선을 추가한다.
frequency = 1000
amplitude = (np.max(vouts) - np.min(vouts)) / 2
sine_wave = -np.sin(2 * np.pi * frequency * times) * amplitude

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, sine_wave, 'Sine Wave', 2)
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.show()

# Calculate the difference between the sine wave and vout
difference = np.abs(sine_wave - vouts)
# Calculate the mean difference
mean_difference_mV = np.mean(difference) * 1000
print_value('mean_difference_mV', mean_difference_mV)

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_1khz_3_9vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# vout과 동일한 주파수의 싸인파 곡선을 추가한다.
frequency = 1000
amplitude = (np.max(vouts) - np.min(vouts)) / 2
sine_wave = -np.sin(2 * np.pi * frequency * times) * amplitude

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, sine_wave, 'Sine Wave', 2)
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.show()

# Calculate the difference between the sine wave and vout
difference = np.abs(sine_wave - vouts)
# Calculate the mean difference
mean_difference_mV = np.mean(difference) * 1000
print_value('mean_difference_mV', mean_difference_mV)

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_1khz_3_8vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# vout과 동일한 주파수의 싸인파 곡선을 추가한다.
frequency = 1000
amplitude = (np.max(vouts) - np.min(vouts)) / 2
sine_wave = -np.sin(2 * np.pi * frequency * times) * amplitude

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, sine_wave, 'Sine Wave', 2)
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.show()

# Calculate the difference between the sine wave and vout
difference = np.abs(sine_wave - vouts)
# Calculate the mean difference
mean_difference_mV = np.mean(difference) * 1000
print_value('mean_difference_mV', mean_difference_mV)

* 고찰:
    + ?? V 부터 왜곡이 나타니자 않음
    + 이상적 파형과의 차이 평균 (mean_difference_mV) 값이 대략 ?? mV 일때를 기준으로 판단했음

*******************************************************************************
## 실험06 CE증폭기회로1의 교류증폭의 민감도 실험

### 실험06-01 동작점 이동에 따른 왜곡 관측 (R1 변경)

#### 실험 방법

* R1에 100 kOhm 대신 68 kOhm을 연결한다.
* Vi의 크기를 변경해 왜곡이 나타나지 않는 시작 전압을 관측한다.
    + 값은 소수점 한자리 단위로 변경

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch07_ep05_02_result_r1_68kohm_1khz_3_9vpp.png", width=600)
display_image("./data/ch07_ep05_02_result_r1_68kohm_1khz_2_2vpp.png", width=600)
display_image("./data/ch07_ep05_02_result_r1_68kohm_1khz_2_1vpp.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_r1_68kohm_1khz_3_9vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# vout과 동일한 주파수의 싸인파 곡선을 추가한다.
frequency = 1000
amplitude = (np.max(vouts) - np.min(vouts)) / 2
sine_wave = -np.sin(2 * np.pi * frequency * times) * amplitude

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, sine_wave, 'Sine Wave', 2)
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.show()

# Calculate the difference between the sine wave and vout
difference = np.abs(sine_wave - vouts)
# Calculate the mean difference
mean_difference_mV = np.mean(difference) * 1000
print_value('mean_difference_mV', mean_difference_mV)

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_r1_68kohm_1khz_2_2vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# vout과 동일한 주파수의 싸인파 곡선을 추가한다.
frequency = 1000
amplitude = (np.max(vouts) - np.min(vouts)) / 2
sine_wave = -np.sin(2 * np.pi * frequency * times) * amplitude

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, sine_wave, 'Sine Wave', 2)
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.show()

# Calculate the difference between the sine wave and vout
difference = np.abs(sine_wave - vouts)
# Calculate the mean difference
mean_difference_mV = np.mean(difference) * 1000
print_value('mean_difference_mV', mean_difference_mV)

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_r1_68kohm_1khz_2_1vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# vout과 동일한 주파수의 싸인파 곡선을 추가한다.
frequency = 1000
amplitude = (np.max(vouts) - np.min(vouts)) / 2
sine_wave = -np.sin(2 * np.pi * frequency * times) * amplitude

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, sine_wave, 'Sine Wave', 2)
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.show()

# Calculate the difference between the sine wave and vout
difference = np.abs(sine_wave - vouts)
# Calculate the mean difference
mean_difference_mV = np.mean(difference) * 1000
print_value('mean_difference_mV', mean_difference_mV)

* 고찰:
    + ?? V 부터 왜곡이 나타니자 않음
    + 이상적 파형과의 차이 평균 (mean_difference_mV) 값이 대략 ?? mV 일때를 기준으로 판단했음

### 실험06-02 동작점 이동에 따른 왜곡 관측 (R2 변경)

#### 실험 방법

* R2에 22 kOhm 대신 15 kOhm을 연결한다.
* Vi의 크기를 변경해 왜곡이 나타나지 않는 시작 전압을 관측한다.
    + 값은 소수점 한자리 단위로 변경

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch07_ep05_02_result_r2_15kohm_1khz_3_9vpp.png", width=600)
display_image("./data/ch07_ep05_02_result_r2_15kohm_1khz_2_7vpp.png", width=600)
display_image("./data/ch07_ep05_02_result_r2_15kohm_1khz_2_6vpp.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_r2_15kohm_1khz_3_9vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# vout과 동일한 주파수의 싸인파 곡선을 추가한다.
frequency = 1000
amplitude = (np.max(vouts) - np.min(vouts)) / 2
sine_wave = -np.sin(2 * np.pi * frequency * times) * amplitude

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, sine_wave, 'Sine Wave', 2)
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.show()

# Calculate the difference between the sine wave and vout
difference = np.abs(sine_wave - vouts)
# Calculate the mean difference
mean_difference_mV = np.mean(difference) * 1000
print_value('mean_difference_mV', mean_difference_mV)

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_r2_15kohm_1khz_2_7vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# vout과 동일한 주파수의 싸인파 곡선을 추가한다.
frequency = 1000
amplitude = (np.max(vouts) - np.min(vouts)) / 2
sine_wave = -np.sin(2 * np.pi * frequency * times) * amplitude

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, sine_wave, 'Sine Wave', 2)
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.show()

# Calculate the difference between the sine wave and vout
difference = np.abs(sine_wave - vouts)
# Calculate the mean difference
mean_difference_mV = np.mean(difference) * 1000
print_value('mean_difference_mV', mean_difference_mV)

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_r2_15kohm_1khz_2_6vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# vout과 동일한 주파수의 싸인파 곡선을 추가한다.
frequency = 1000
amplitude = (np.max(vouts) - np.min(vouts)) / 2
sine_wave = -np.sin(2 * np.pi * frequency * times) * amplitude

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, sine_wave, 'Sine Wave', 2)
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.show()

# Calculate the difference between the sine wave and vout
difference = np.abs(sine_wave - vouts)
# Calculate the mean difference
mean_difference_mV = np.mean(difference) * 1000
print_value('mean_difference_mV', mean_difference_mV)

* 고찰:
    + ?? V 부터 왜곡이 나타니자 않음
    + 이상적 파형과의 차이 평균 (mean_difference_mV) 값이 대략 ?? mV 일때를 기준으로 판단했음

*******************************************************************************
## 실험07 부하효과의 심각성을 보자

#### 실험 방법

* 부하저항 RO가 무한대(부하 없음), 200 Ohm (부하 큼) 일때 출력을 관측한다.

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch07_ep05_02_result_ro_infinite_1khz_2vpp.png", width=600)
display_image("./data/ch07_ep05_02_result_ro_200ohm_1khz_2vpp.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_ro_infinite_1khz_2vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.ylim(-4, 4)
plt.show()

V_O_pp_V = vouts.max() - vouts.min()
print_value("V_O_pp_V", V_O_pp_V)

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch07_ep05_02_result_ro_200ohm_1khz_2vpp.csv')

# 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
times = data['TIME']
vins = data['Sample CH1']
vouts = data['Sample CH2']

# 필터를 적용해 잡음을 줄인다.
vins = savgol_filter(vins, 100, 2)
vouts = savgol_filter(vouts, 100, 2)

# 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
plt.xlabel('TIME_s')
plt.ylabel('V_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vouts, 'V(OUT)', 1)
plt.xlim(0.000, 0.001)
plt.ylim(-4, 4)
plt.show()

V_O_pp_V = vouts.max() - vouts.min()
print_value("V_O_pp_V", V_O_pp_V)

* 고찰:
    + 부하 저항이 ??면 (부하가 ??면) 출력이 ??한다.

*******************************************************************************
## 실험08 출력저항 $R_O$의 측정

#### 실험 방법

* 부하저항 RO에 5 kOhm 가변저항을 연결한다.
* RO 값을 변화시켜 출력전압을 부하 저항이 무한대(부하 없음) 일때의 절반으로 감소시킨다.
* 가변저항을 분리해 멀티미터로 저항값을 측정한다.

#### 실험 결과

Ro = ?? kOhm

* 고찰:
    + 이론적인 Ro 값과 ??.